In [1]:
import os
import pandas as pd
import numpy as np
import warnings
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from dotenv import load_dotenv

# Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Data Encoding and Scaling
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

# Pipeline
from sklearn.pipeline import Pipeline
import pickle

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# Loading .env file into my python code
load_dotenv()

True

In [4]:
def create_connection():
    print('creating connection with DB')
    user = os.getenv("DB_USER")
    raw_password = os.getenv("DB_PASSWORD")
    password = quote_plus(raw_password)
    host = os.getenv("DB_HOST")
    port = os.getenv("DB_PORT")
    db = os.getenv("DB_NAME")

    # Credentials of mySQL connection
    connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}"
    engine = create_engine(connection_string)
    print('connection created successfully')
    return engine

In [5]:
def create_df(engine,catcher_only=False):
    print('creating DFs=============')
    if catcher_only:
        print('Catcher DF is requested')
        catcher_df = pd.read_sql('Select jobma_catcher_id, is_premium, jobma_catcher_parent, jobma_verified, subscription_status, company_size FROM jobma_catcher', con=engine) 
        return catcher_df
    print("Wallet DF")
    wallet_df = pd.read_sql('Select catcher_id AS jobma_catcher_id, is_unlimited FROM wallet', con=engine)
    print("Subscription DF")
    subscription_df = pd.read_sql('Select catcher_id AS jobma_catcher_id, currency, subscription_amount FROM subscription_history', con=engine)
    print("Invitation DF")
    invitation_df = pd.read_sql('Select jobma_catcher_id, jobma_interview_mode, jobma_interview_status FROM jobma_pitcher_invitations', con=engine)
    print("Job posting DF")
    job_posting_df = pd.read_sql('Select jobma_catcher_id FROM jobma_employer_job_posting', con=engine)
    print("kit DF")
    kit_df = pd.read_sql('Select catcher_id AS jobma_catcher_id FROM job_assessment_kit', con=engine)
    print('Login DF')
    login_df = pd.read_sql('Select jobma_role_id, jobma_user_id AS jobma_catcher_id, jobma_last_login FROM jobma_login',con=engine)
    
    # Closing the Connection
    engine.dispose()
    return wallet_df,subscription_df,invitation_df,job_posting_df,kit_df,login_df

# Specific Methods

**To Fetch Columns from different tables and fitting those functions into Pipeline**

In [7]:
# catcher_df

def fetching_catcher_df(catcher_df):
    print("Processing catcher DF")
    catcher_df['jobma_verified'] = catcher_df['jobma_verified'].replace({'0':0, '1':1})
    catcher_df.drop(catcher_df[catcher_df['is_premium'] == ''].index, inplace=True)
    catcher_df['is_premium'] = catcher_df['is_premium'].replace({'0':0, '1':1})
    catcher_df['subscription_status'] = catcher_df['subscription_status'].replace({'0':0, '1':1, '2':0})

    return catcher_df

In [8]:
# wallet_df

def fetching_wallet_df(wallet_df):
    print("Processing wallet DF")
    wallet_df['is_unlimited'] = wallet_df['is_unlimited'].replace({'0':0, '1':1})
    wallet_df = wallet_df[wallet_df['is_unlimited'] != '']
    wallet_df.drop_duplicates(inplace=True)

    return wallet_df

In [9]:
# subscription_df

def fetching_subscription_df(subscription_df):
    print("Processing subscription DF")
    subscription_df.loc[subscription_df['currency'] == '1', 'subscription_amount'] /= 85.23
    subscription_df = subscription_df.groupby('jobma_catcher_id').agg(
        subscription_amount_in_dollars = ('subscription_amount', 'sum'),
        number_of_subscriptions = ('subscription_amount', 'count'),
    ).reset_index()
    subscription_df.drop_duplicates(inplace=True)
    return subscription_df

In [10]:
# login_df

def fetching_login_df(login_df):
    print("Processing login DF")
    login_df = login_df[login_df['jobma_role_id'] == 3].copy()

    # Calculating Number of Gaps between last login and today
    login_df['jobma_last_login'] = pd.to_datetime(login_df['jobma_last_login'], errors='coerce')
    login_df['activity_duration'] = (pd.Timestamp('today') - login_df['jobma_last_login']).dt.days
    login_df['activity_duration'].fillna(5, inplace=True)
    login_df['activity_duration'] = login_df['activity_duration'].astype(int)

    # Binning
    bins = [-1,7,30,90,180,365,float('inf')]
    labels = ['Less than 1 Week', '1-4 Weeks', '1-3 Months', '3-6 Months', '6-12 Months', 'More than 1 Year']
    login_df['activity_duration'] = pd.cut(login_df['activity_duration'], bins=bins, labels=labels)
    login_df = login_df[['jobma_catcher_id', 'activity_duration']]

    return login_df

In [11]:
def fetching_features(invitation_df, job_posting_df, kit_df):
    print("Fetching features")

    job_posting_df['job_posted'] = job_posting_df['jobma_catcher_id'].map(job_posting_df['jobma_catcher_id'].value_counts())
    kit_df['number_of_kits'] = kit_df['jobma_catcher_id'].map(kit_df['jobma_catcher_id'].value_counts())

    invitation_df = invitation_df[invitation_df['jobma_interview_status'] == '2']
    invitation_df['number_of_invitations'] = invitation_df['jobma_catcher_id'].map(invitation_df['jobma_catcher_id'].value_counts())
    invitation_df.drop('jobma_interview_status', axis=1, inplace=True)
    invitation_df['interview_completed'] = invitation_df['jobma_catcher_id'].map(invitation_df['jobma_catcher_id'].value_counts())
    invitation_df = invitation_df[invitation_df['jobma_interview_mode'].isin(['1', '2'])].copy()
    interview_counts = invitation_df.groupby(['jobma_catcher_id', 'jobma_interview_mode']).size().unstack(fill_value=0)
    interview_counts = interview_counts.rename(columns={'1': 'number_of_recorded_interviews', '2': 'number_of_live_interviews'})
    invitation_df = invitation_df.merge(interview_counts, on='jobma_catcher_id', how='left')
    
    invitation_df.drop('jobma_interview_mode', axis=1, inplace=True)
    invitation_df = invitation_df.drop_duplicates()

    job_posting_df = job_posting_df[['jobma_catcher_id', 'job_posted']].drop_duplicates()
    kit_df = kit_df[['jobma_catcher_id', 'number_of_kits']].drop_duplicates()

    return invitation_df, job_posting_df, kit_df

In [12]:
login_order = {
        'Less than 1 Week':0,
        '1-4 Weeks':1,
        '1-3 Months':2,
        '3-6 Months':3,
        '6-12 Months':4,
        'More than 1 Year':5
    }

In [13]:
def merging_df(catcher_df, wallet_df, subscription_df, invitation_df, job_posting_df, kit_df, login_df):
    print("Merging DFs")
    final_df = catcher_df.copy()

    # Left join each table one by one
    final_df = final_df.merge(wallet_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(subscription_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(invitation_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(job_posting_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(kit_df, on='jobma_catcher_id', how='left')
    final_df = final_df.merge(login_df, on='jobma_catcher_id', how='left')
    final_df.drop_duplicates(inplace=True)

    # For Total Sub
    sub_counts = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent').size()
    final_df['total_sub'] = final_df['jobma_catcher_id'].map(sub_counts).fillna(0).astype(int)

    # For Kits
    sub_kits_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['number_of_kits'].sum()
    kits_mapped = final_df['jobma_catcher_id'].map(sub_kits_sum).fillna(0)
    final_df['number_of_kits'] = final_df['number_of_kits'].fillna(0) + kits_mapped
    final_df['number_of_kits'] = final_df['number_of_kits'].astype(int)
    
    # For Invitations
    sub_invitations_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['number_of_invitations'].sum()
    invitations_mapped = final_df['jobma_catcher_id'].map(sub_invitations_sum).fillna(0)
    final_df['number_of_invitations'] = final_df['number_of_invitations'].fillna(0) + invitations_mapped
    final_df['number_of_invitations'] = final_df['number_of_invitations'].astype(int)
    
    # For Job Posted
    sub_job_posted_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['job_posted'].sum()
    job_posted_mapped = final_df['jobma_catcher_id'].map(sub_job_posted_sum).fillna(0)
    final_df['job_posted'] = final_df['job_posted'].fillna(0) + job_posted_mapped
    final_df['job_posted'] = final_df['job_posted'].astype(int)

    # For Recorded Interviews
    sub_recorded_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['number_of_recorded_interviews'].sum()
    recorded_mapped = final_df['jobma_catcher_id'].map(sub_recorded_sum).fillna(0)
    final_df['number_of_recorded_interviews'] = final_df['number_of_recorded_interviews'].fillna(0) + recorded_mapped
    final_df['number_of_recorded_interviews'] = final_df['number_of_recorded_interviews'].astype(int)

    # For Live Interviews
    sub_live_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['number_of_live_interviews'].sum()
    live_mapped = final_df['jobma_catcher_id'].map(sub_live_sum).fillna(0)
    final_df['number_of_live_interviews'] = final_df['number_of_live_interviews'].fillna(0) + live_mapped
    final_df['number_of_live_interviews'] = final_df['number_of_live_interviews'].astype(int)

    # For Interview Completed
    sub_to_parent_sum = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['interview_completed'].sum()
    final_df.loc[final_df['jobma_catcher_id'].isin(sub_to_parent_sum.index), 'interview_completed'] += final_df['jobma_catcher_id'].map(sub_to_parent_sum).fillna(0).astype(int)

    # For Minimum Login Days
    # login_order = {
    #     'Less than 1 Week':0,
    #     '1-4 Weeks':1,
    #     '1-3 Months':2,
    #     '3-6 Months':3,
    #     '6-12 Months':4,
    #     'More than 1 Year':5
    # }

    final_df['activity_duration'] = final_df['activity_duration'].map(login_order).fillna(5).astype(int)
    sub_min_login = final_df[final_df['jobma_catcher_parent'] != 0].groupby('jobma_catcher_parent')['activity_duration'].min()
    final_df.loc[final_df['jobma_catcher_id'].isin(sub_min_login.index), 'activity_duration'] = final_df.loc[final_df['jobma_catcher_id'].isin(sub_min_login.index), 'jobma_catcher_id'].map(sub_min_login)

    verified_df = final_df[final_df['jobma_verified'] == 1].copy()
    df = verified_df[verified_df['jobma_catcher_parent'] == 0].copy()
    df.drop(['jobma_catcher_parent', 'jobma_verified'], axis=1, inplace=True)
    
    compare_df = df.copy()
    df.drop('jobma_catcher_id', axis=1, inplace=True)

    print(f"Final merged df shape is {df.shape}")

    return df, compare_df

In [14]:
def fill_missing_values(final_df):
    final_df = final_df.copy()

    # Step 1: Replace inf with NaN first
    final_df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Step 2: Fill NaNs
    fill_values = {
        'is_premium': 0,
        'subscription_status': 0,
        'company_size': '1-25',
        'is_unlimited': 0,
        'subscription_amount_in_dollars': 0,
        'number_of_subscriptions': 0,
        'number_of_invitations': 0,
        'interview_completed': 0,
        'number_of_recorded_interviews': 0,
        'number_of_live_interviews': 0,
        'job_posted': 0,
        'number_of_kits': 0,
        'activity_duration': 5,
        'total_sub': 0,
    }
    final_df.fillna(fill_values, inplace=True)

    # Step 3: Explicitly cast to int for the appropriate columns
    int_columns = [
        'is_premium',
        'subscription_status',
        'is_unlimited',
        'number_of_subscriptions',
        'jobma_interview_status',
        'number_of_invitations',
        'interview_completed',
        'number_of_recorded_interviews',
        'number_of_live_interviews',
        'job_posted',
        'number_of_kits',
        'activity_duration',
        'total_sub',
    ]
    
    for col in int_columns:
        if col in final_df.columns:
            final_df[col] = pd.to_numeric(final_df[col], errors='coerce').fillna(0).astype(int)

    return final_df

In [15]:
# Data Encoding 

def ordinal_encoder(df):
    ordinal_col = ['company_size']
    company_size_order = ['1-25', '26-100', '101-500', '500-1000', 'More than 1000']

    ordinal = OrdinalEncoder(categories=[company_size_order])

    encoded = ordinal.fit_transform(df[ordinal_col].astype(str))

    encoded_df = pd.DataFrame(encoded, columns=[f' {col}_ord' for col in ordinal_col], index=df.index)

    df.drop(columns=ordinal_col, inplace=True)

    df = pd.concat([df, encoded_df], axis=1)

    return df

In [16]:
# Log Transformation 

def log_transform(df):
    log_cols = [
        'subscription_amount_in_dollars',
        'number_of_subscriptions',
        'interview_completed',
        'number_of_invitations',
        'number_of_recorded_interviews',
        'number_of_live_interviews',
        'job_posted',
        'number_of_kits',
        'activity_duration'
        'total_sub'
    ]

    df = df.copy()
    for col in log_cols:
        if col in df.columns:
            # fill NaNs
            df[col] = df[col].fillna(0)
            # if a number is less than zero, turn it into zero
            df[col] = df[col].clip(lower=0)
            # safe log1p
            df[col] = np.log1p(df[col])

    return df

In [17]:
def to_tensor(x):
    return torch.tensor(x, dtype=torch.float32)

# Pipeline

In [19]:
class MergingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        engine = create_connection()
        wallet_df,subscription_df,invitation_df,job_posting_df,kit_df, login_df = create_df(engine)
        self.wallet_df = wallet_df
        self.subscription_df = subscription_df
        self.invitation_df = invitation_df
        self.job_posting_df = job_posting_df
        self.kit_df = kit_df
        self.login_df = login_df

    def fit(self, X, y=None):
        return self

    def transform(self, catcher_df):
        catcher = fetching_catcher_df(catcher_df)
        wallet = fetching_wallet_df(self.wallet_df)
        subscription = fetching_subscription_df(self.subscription_df)
        login = fetching_login_df(self.login_df)
        invitation, job_posting, kit = fetching_features(
            self.invitation_df,
            self.job_posting_df,
            self.kit_df,
        )
        final_df, compare_df = merging_df(catcher, wallet, subscription, invitation, job_posting, kit, login)
        self.compare_df_ = fill_missing_values(compare_df)
        self.compare_df_.to_csv("compare_df.csv", index=False)
        return final_df

In [20]:
class PreprocessingTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = fill_missing_values(X)
        X = ordinal_encoder(X)
        X = log_transform(X)
        return X

In [21]:
class AutoEncoder(nn.Module):
    def __init__(self, input_shape):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_shape, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)  # bottleneck
        )

        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, input_shape)
        )

    def forward(self, X):
        encoded = self.encoder(X)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [22]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [23]:
def train_and_save_model():
    print("Starting with model training")
    
    # Pipelines
    merge_pipeline = Pipeline([
        ('merge', MergingTransformer())
    ])

    # Step 2: Preprocess merged data
    preprocess_pipeline = Pipeline([
        ('preprocessing', PreprocessingTransformer()),
        ('scaler', StandardScaler()),
        ('to_tensor', FunctionTransformer(to_tensor))
    ])

    full_pipeline = Pipeline([
        ('merge_pipeline', merge_pipeline),
        ('preprocess_pipeline', preprocess_pipeline)
    ])

    X_tensor = full_pipeline.fit_transform(create_df(create_connection(),catcher_only=True))
    
    with open('full_pipeline.pkl', 'wb') as f:
        pickle.dump(full_pipeline, f)
    
    X_df = pd.DataFrame(X_tensor)
    X_data = CustomDataset(X_tensor)

    BATCH_SIZE = 16
    dataloader = DataLoader(X_data, batch_size=BATCH_SIZE, shuffle=True)

    # Initializing the Model 
    input_shape = X_df.shape[1]
    model_1 = AutoEncoder(input_shape)

    # Important Parameters 
    learning_rate = 0.001
    epochs = 50
    patience = 5
    delta = 1e-4
    best_loss = float('inf')
    epochs_no_improve = 0
    training_losses = []

    # Loss Function, Optimizers and LR Scheduler 
    loss_function = nn.SmoothL1Loss()
    optimizer = torch.optim.AdamW(model_1.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

    for epoch in range(epochs):
        model_1.train()
        epoch_loss = 0

        for batch in dataloader:
            encoded, decoded = model_1(batch)
            loss = loss_function(decoded, batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        training_losses.append(avg_loss)
        scheduler.step(avg_loss)

        print(f"Epoch: {epoch+1}/{epochs} | Loss: {avg_loss:.6f}")

        # Early Stopping
        if avg_loss < best_loss - delta:
            best_loss = avg_loss
            epochs_no_improve = 0

        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

        # Print current learning rate
        for param_group in optimizer.param_groups:
            print(f"Learning Rate: {param_group['lr']:.6f}")
    print("Saving Model and Embeddings")
    torch.save(model_1.state_dict(),'model.pth')
    encoder, _ = model_1(X_tensor)
    latent_np = encoder.cpu().detach().numpy()

    with open('latent_np.pkl', 'wb') as embeddings_file:
        pickle.dump(latent_np, embeddings_file)

    print("Model and Embeddings Saved")

In [24]:
# Expecting catcher_id to fetch all the details of user

def predict_using_catcher_id(catcher_id, n=5):
    with open('full_pipeline.pkl', 'rb') as f:
        full_pipeline = pickle.load(f)

    # Extract Pipelines
    preprocess_pipeline = full_pipeline.named_steps['preprocess_pipeline']
    compare_df = pd.read_csv('compare_df.csv')

    user_row = compare_df[compare_df['jobma_catcher_id'] == catcher_id]

    if user_row.empty:
        raise ValueError(f"No data found for catcher_id: {catcher_id}")

    user_input = user_row.drop(columns=['jobma_catcher_id']).iloc[0].to_dict()
    user_df = pd.DataFrame([user_input])
    transformed_input = preprocess_pipeline.transform(user_df)
    print(f'Transformed User Input Type is: {type(transformed_input)}')

    # Load trained model
    input_shape = transformed_input.shape[1]
    model_1 = AutoEncoder(input_shape)
    model_1.load_state_dict(torch.load('model.pth'))
    model_1.eval()

     # Load latent embeddings
    with open('latent_np.pkl', 'rb') as embeddings_file:
        embeddings = pickle.load(embeddings_file)
    print(f'Embedding Type is {type(embeddings)}')
    
    # Generate user embedding
    with torch.no_grad():
        user_embedding, _ = model_1(transformed_input)
        user_embedding = F.normalize(user_embedding, p=2, dim=1)
        print(f'User Embedding Type is: {type(user_embedding)}')

        latent_embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)
        latent_embeddings = F.normalize(latent_embeddings_tensor, p=2, dim=1)
        print(f'Latent Embedding Type is {type(latent_embeddings)}')

        # Compute cosine similarity and get top-N
        similarities = F.cosine_similarity(user_embedding, latent_embeddings, dim=1)
        top_indices = similarities.topk(n).indices.cpu().numpy()

    # Final Recommendation DataFrame
    recommended = compare_df.iloc[top_indices].copy()
    
    # To show the actual activity duration
    recommended['activity_duration'] = recommended['activity_duration'].replace({v:k for k,v in login_order.items()})

    # To show the Similarity Score
    recommended['similarity'] = similarities[top_indices].cpu().numpy()

    return recommended

In [25]:
# Expecting all details from user_input

def predict(user_input, n=5):
    with open('full_pipeline.pkl', 'rb') as f:
        full_pipeline = pickle.load(f)

    # Extract pipelines
    preprocess_pipeline = full_pipeline.named_steps['preprocess_pipeline']

    # access compare_df using .csv file instead of fetching it from database
    compare_df = pd.read_csv('compare_df.csv')

    # Transform user input only with preprocessing pipeline
    user_df = pd.DataFrame([user_input])
    transformed_input = preprocess_pipeline.transform(user_df)
    print(f'Transformed User Input Type is: {type(transformed_input)}')

    # Load trained model
    input_shape = transformed_input.shape[1]
    model_1 = AutoEncoder(input_shape)
    model_1.load_state_dict(torch.load('model.pth'))
    model_1.eval()

    # Load latent embeddings
    with open('latent_np.pkl', 'rb') as embeddings_file:
        embeddings = pickle.load(embeddings_file)
    print(f'Embedding Type is {type(embeddings)}')
    
    # Generate user embedding
    with torch.no_grad():
        user_embedding, _ = model_1(transformed_input)
        user_embedding = F.normalize(user_embedding, p=2, dim=1)
        print(f'User Embedding Type is: {type(user_embedding)}')

        latent_embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)
        latent_embeddings = F.normalize(latent_embeddings_tensor, p=2, dim=1)
        print(f'Latent Embedding Type is {type(latent_embeddings)}')

        # Compute cosine similarity and get top-N
        similarities = F.cosine_similarity(user_embedding, latent_embeddings, dim=1)
        top_indices = similarities.topk(n).indices.cpu().numpy()
    
    # Final Recommendation DataFrame
    recommended = compare_df.iloc[top_indices].copy()
    
    # To show the actual activity duration
    recommended['activity_duration'] = recommended['activity_duration'].replace({v:k for k,v in login_order.items()})

    # To show the Similarity Score
    recommended['similarity'] = similarities[top_indices].cpu().numpy()

    return recommended

In [26]:
user_pref_good = {'is_premium':0,
                 'subscription_status':0,
                 'company_size':'101-500', 
                 'is_unlimited':0,
                 'subscription_amount_in_dollars': 100.00,
                 'number_of_subscriptions':1,
                 'number_of_invitations':25,
                'interview_completed':10,
                'number_of_recorded_interviews':8,
                'number_of_live_interviews':5,
                'job_posted':4,
                'number_of_kits':7,
                'activity_duration':4,
                'total_sub':1,
            }

In [27]:
user_pref_test = {'is_premium':1,
                  'subscription_status':1,
                'company_size':'1-25',
                 'is_unlimited':1,
                 'subscription_amount_in_dollars': 125.0,
                 'number_of_subscriptions':1,
                 'number_of_invitations':18,
                 'interview_completed':10,
                  'number_of_recorded_interviews':523,
                'number_of_live_interviews':1,
                 'job_posted':1,
                 'number_of_kits':1,
                'activity_duration':5,
                'total_sub':1
            }

In [28]:
train_and_save_model()

Starting with model training
creating connection with DB
connection created successfully
creating DFs=============
Wallet DF
Subscription DF
Invitation DF
Job posting DF
kit DF
Login DF
creating connection with DB
connection created successfully
creating DFs=============
Catcher DF is requested
Processing catcher DF
Processing wallet DF
Processing subscription DF
Processing login DF
Fetching features
Merging DFs
Final merged df shape is (4463, 14)
Epoch: 1/50 | Loss: 0.161954
Learning Rate: 0.001000
Epoch: 2/50 | Loss: 0.070442
Learning Rate: 0.001000
Epoch: 3/50 | Loss: 0.057293
Learning Rate: 0.001000
Epoch: 4/50 | Loss: 0.050645
Learning Rate: 0.001000
Epoch: 5/50 | Loss: 0.045861
Learning Rate: 0.001000
Epoch: 6/50 | Loss: 0.042468
Learning Rate: 0.001000
Epoch: 7/50 | Loss: 0.042053
Learning Rate: 0.001000
Epoch: 8/50 | Loss: 0.039875
Learning Rate: 0.001000
Epoch: 9/50 | Loss: 0.038082
Learning Rate: 0.001000
Epoch: 10/50 | Loss: 0.034465
Learning Rate: 0.001000
Epoch: 11/50 | Lo

In [29]:
predict(user_pref_good, 5)

Transformed User Input Type is: <class 'torch.Tensor'>
Embedding Type is <class 'numpy.ndarray'>
User Embedding Type is: <class 'torch.Tensor'>
Latent Embedding Type is <class 'torch.Tensor'>


,jobma_catcher_id,is_premium,subscription_status,company_size,is_unlimited,subscription_amount_in_dollars,number_of_subscriptions,number_of_invitations,interview_completed,number_of_recorded_interviews,number_of_live_interviews,job_posted,number_of_kits,activity_duration,total_sub,similarity
315,3394,0,0,26-100,0,212.844890,2,9,9,1,8,4,3,More than 1 Year,1,0.903045
1906,5921,0,0,26-100,0,5.524111,1,2,0,2,0,2,1,More than 1 Year,1,0.892414
1566,5389,0,0,1-25,0,499.000000,1,8,8,8,0,11,2,More than 1 Year,1,0.891605
3757,8736,0,0,26-100,0,150.000000,1,21,21,21,0,7,5,More than 1 Year,4,0.882265
4426,10482,0,1,101-500,0,100.000000,1,14,14,9,5,4,7,More than 1 Year,1,0.882167


In [30]:
predict(user_pref_test, 5)

Transformed User Input Type is: <class 'torch.Tensor'>
Embedding Type is <class 'numpy.ndarray'>
User Embedding Type is: <class 'torch.Tensor'>
Latent Embedding Type is <class 'torch.Tensor'>


,jobma_catcher_id,is_premium,subscription_status,company_size,is_unlimited,subscription_amount_in_dollars,number_of_subscriptions,number_of_invitations,interview_completed,number_of_recorded_interviews,number_of_live_interviews,job_posted,number_of_kits,activity_duration,total_sub,similarity
4156,10065,0,1,1-25,1,0.000000,1,44,44,44,0,3,2,More than 1 Year,0,0.943716
4005,9794,0,1,26-100,0,50.000000,1,24,24,24,0,1,6,More than 1 Year,1,0.886676
4095,9947,0,1,1-25,0,200.000000,2,17,17,17,0,3,5,More than 1 Year,1,0.850046
2950,7399,0,0,1-25,1,878.798545,4,206,206,206,0,2,1,More than 1 Year,2,0.841861
2022,6110,0,0,1-25,0,207.673354,1,179,179,179,0,1,3,More than 1 Year,1,0.838338


In [31]:
predict_using_catcher_id(6025, 5)

Transformed User Input Type is: <class 'torch.Tensor'>
Embedding Type is <class 'numpy.ndarray'>
User Embedding Type is: <class 'torch.Tensor'>
Latent Embedding Type is <class 'torch.Tensor'>


,jobma_catcher_id,is_premium,subscription_status,company_size,is_unlimited,subscription_amount_in_dollars,number_of_subscriptions,number_of_invitations,interview_completed,number_of_recorded_interviews,number_of_live_interviews,job_posted,number_of_kits,activity_duration,total_sub,similarity
1970,6025,1,0,26-100,0,399.000000,1,25,25,25,0,2,8,More than 1 Year,1,0.985181
2078,6183,1,0,26-100,0,761.468966,3,16,16,16,0,4,5,More than 1 Year,1,0.976384
1877,5856,1,0,26-100,0,415.346709,2,104,104,104,0,12,8,More than 1 Year,4,0.967532
1896,5898,1,0,26-100,0,2076.733545,6,436,436,436,0,20,78,More than 1 Year,4,0.966576
3768,8751,1,0,26-100,0,1850.000000,2,17,17,17,0,1,3,More than 1 Year,1,0.962689


In [32]:
predict_using_catcher_id(6189, 5)

Transformed User Input Type is: <class 'torch.Tensor'>
Embedding Type is <class 'numpy.ndarray'>
User Embedding Type is: <class 'torch.Tensor'>
Latent Embedding Type is <class 'torch.Tensor'>


,jobma_catcher_id,is_premium,subscription_status,company_size,is_unlimited,subscription_amount_in_dollars,number_of_subscriptions,number_of_invitations,interview_completed,number_of_recorded_interviews,number_of_live_interviews,job_posted,number_of_kits,activity_duration,total_sub,similarity
2083,6189,1,0,1-25,0,346.122257,1,177,177,177,0,45,23,More than 1 Year,0,0.965194
2243,6427,1,0,1-25,0,175.994368,1,34,34,34,0,3,3,More than 1 Year,2,0.961727
1881,5875,1,0,1-25,0,207.673354,1,15,15,15,0,6,5,More than 1 Year,2,0.948215
1574,5404,1,0,26-100,0,207.673354,1,8,8,8,0,4,2,More than 1 Year,0,0.946078
1970,6025,1,0,26-100,0,399.000000,1,25,25,25,0,2,8,More than 1 Year,1,0.944187
